In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import GradientBoostingRegressor
from IPython.display import display
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
energy_train = pd.read_parquet('energy_train.parquet')
energy_test2 = pd.read_parquet('energy_test2.parquet')
energy_test1 = pd.read_parquet('energy_test1.parquet')
forecasts = pd.read_parquet('forecasts.parquet')

energy_test1_copy = pd.read_parquet('energy_test1.parquet')
energy_test1.head()

### Zusammenführen der Wettermodelle

In [ ]:
for model in ['DWD ICON', 'NCEP GFS']:
    # Filter für das Wettermodell
    forecasts_model = forecasts[forecasts['Weather Model'] == model].copy()
    
    # Spalten umbenennen
    forecasts_model = forecasts_model.rename(columns={
        'SolarDownwardRadiation': f'SolarDownwardRadiation_{model.replace(" ", "_")}',
        'CloudCover': f'CloudCover_{model.replace(" ", "_")}',
        'Temperature': f'Temperature_{model.replace(" ", "_")}'
    })
    
    # 'valid_datetime' berechnen
    forecasts_model['valid_datetime'] = pd.to_datetime(forecasts_model['ref_datetime']) + pd.to_timedelta(forecasts_model['valid_time'], unit='h')
    
    # Forecast DataFrame für das spezifische Modell speichern
    if model == 'DWD ICON':
        forecasts_dwd = forecasts_model
    else:
        forecasts_ncep = forecasts_model

# Zusammenführen der beiden Modelle
forecasts_combined = pd.merge(
    forecasts_ncep,
    forecasts_dwd, 
    on=['ref_datetime', 'valid_time', 'valid_datetime'], 
    how='inner'
)

forecasts_combined.head()


### Zusammenführen der Energiedaten und Wettervorhersagen

In [ ]:
# Merging beide DataFrames basierend auf den Spalten 'dtm' und 'ref_datetime' in energy_train
# sowie 'valid_datetime' und 'ref_datetime' in forecasts_combined (inner join).
energy_train_mit_forecast = pd.merge(
    energy_train, 
    forecasts_combined, 
    left_on=['dtm', 'ref_datetime'], 
    right_on=['valid_datetime', 'ref_datetime'], 
    how='inner'
)

# Entferne Zeilen, bei denen die Zielvariable 'Solar_MWh' NaN ist.
energy_train_mit_forecast = energy_train_mit_forecast[energy_train_mit_forecast["Solar_MWh"].isna() == False]

energy_train_mit_forecast.head()


In [ ]:
# Extrahiere Monat und Jahr aus der Spalte 'dtm' im Format "Monat Jahr"
energy_train_mit_forecast['month_year'] = energy_train_mit_forecast['dtm'].dt.strftime('%B %Y')

# Erhalte eindeutige Werte der Monate und Jahre
unique_months_years = energy_train_mit_forecast['month_year'].unique()

print(unique_months_years)


###  Generieren neuer Features

Korrelation mit Solar_MWh(Aus Aufgabe 4)

| Feature                                | Korrelation |
|----------------------------------------|-------------|
| SolarDownwardRadiation_DWD_ICON        | 0.952156    |
| SolarDownwardRadiation_NCEP_GFS        | 0.933041    |
| radiation_temp_interaction_DWD_ICON   | 0.866000    |
| radiation_temp_interaction_NCEP_GFS   | 0.853576    |
| CloudRadiationLoss                     | 0.828186    |
| effective_radiation_NCEP_GFS          | 0.779750    |
| effective_radiation_DWD_ICON          | 0.724797    |
   

In [ ]:
energy_train_mit_forecast['time'] = energy_train_mit_forecast['dtm'].dt.time  #Stunden extrahieren 
energy_train_mit_forecast['CloudRadiationLoss'] =(energy_train_mit_forecast['SolarDownwardRadiation_DWD_ICON']* energy_train_mit_forecast['CloudCover_DWD_ICON'])
energy_train_mit_forecast['effective_radiation_DWD_ICON'] = energy_train_mit_forecast['SolarDownwardRadiation_DWD_ICON'] * (1 - energy_train_mit_forecast['CloudCover_DWD_ICON'])
energy_train_mit_forecast['effective_radiation_NCEP_GFS'] = energy_train_mit_forecast['SolarDownwardRadiation_NCEP_GFS'] * (1 - energy_train_mit_forecast['CloudCover_NCEP_GFS'])
energy_train_mit_forecast['radiation_temp_interaction_DWD_ICON'] = energy_train_mit_forecast['SolarDownwardRadiation_DWD_ICON'] * energy_train_mit_forecast['Temperature_DWD_ICON'] 
energy_train_mit_forecast['radiation_temp_interaction_NCEP_GFS'] = energy_train_mit_forecast['SolarDownwardRadiation_NCEP_GFS'] * energy_train_mit_forecast['Temperature_NCEP_GFS'] 
energy_train_mit_forecast.head()


### Erstellen von Trainings- und Validierungsset

In [ ]:
# Extrahiere Monat und Jahr zur Filterung
energy_train_mit_forecast['month_year'] = energy_train_mit_forecast['dtm'].dt.to_period('M')

# Teile die Daten in Trainings- und Validierungssets auf
# Das Trainings-Set umfasst die Daten mit `month_year` zwischen September 2020 und Juni 2022 (inklusive).
df_train = energy_train_mit_forecast[(energy_train_mit_forecast['month_year'] >= '2020-09') & (energy_train_mit_forecast['month_year'] <= '2022-06')]

# Das Test-Set umfasst die Daten mit `month_year` nach Juni 2022.
df_test = energy_train_mit_forecast[energy_train_mit_forecast['month_year'] > '2022-06']


### Zu entfernende Spalten

In [ ]:
# Liste der zu entfernenden Spalten
columns_to_drop = ["dtm", "ref_datetime", "Weather Model_x", "Weather Model_y", "valid_datetime", "valid_time", 'month_year',
                   "CloudCover_NCEP_GFS", "CloudCover_DWD_ICON", "Temperature_NCEP_GFS", "Temperature_DWD_ICON", "Solar_capacity_mwp"] 

# Entfernen der definierten Spalten
df_train = df_train.drop(columns=columns_to_drop)


In [ ]:
categorical_features = ["time"] 
numerical_features = ["SolarDownwardRadiation_DWD_ICON", "SolarDownwardRadiation_NCEP_GFS", "CloudRadiationLoss", "effective_radiation_DWD_ICON",
                     "effective_radiation_NCEP_GFS", "radiation_temp_interaction_DWD_ICON", "radiation_temp_interaction_NCEP_GFS"  ]

In [ ]:
# Entfernen des Labels aus den Daten
y_train= df_train.pop("Solar_MWh")
y_test= df_test.pop("Solar_MWh")

In [ ]:
X_train = df_train
X_test = df_test

In [ ]:
X_train.info()

### Columntransformer and pipeline

In [ ]:
# Ziel: Vorverarbeitung der Daten (kategoriale und numerische Features) in separaten Pipelines.
column_trans = ColumnTransformer(
    transformers=[
        # 1. Vorverarbeitung der kategorialen Features:
        #    a) Fehlende Werte (NaN) in kategorialen Spalten durch die häufigste Kategorie ersetzen ("most_frequent").
        #    b) One-hot Encoding für kategoriale Features, um sie in numerische Werte zu transformieren.
        ("onehot", Pipeline([
            ("impute", SimpleImputer(strategy="most_frequent")),  # Fehlende Werte ersetzen
            ("encode", OneHotEncoder(handle_unknown="ignore"))  # One-hot Encoding
        ]), categorical_features),  # Liste der kategorialen Features
        
        # 2. Vorverarbeitung der numerischen Features:
        #    a) Skalierung der numerischen Spalten, um sie standardisiert (Mittelwert=0, Varianz=1) darzustellen.
        ("impute_scale", Pipeline([
            ("impute", SimpleImputer(strategy="mean")),   # Fehlende Werte ersetzen
            ("scale", StandardScaler())  # Skalierung der numerischen Daten
        ]), numerical_features)  # Liste der numerischen Features
    ],
    # Alle anderen Spalten werden unverändert beibehalten (falls vorhanden), da `remainder="passthrough"` angegeben ist.
    remainder="passthrough"
)


## Aufgabe 5

### Einfache lineare Regression

In [ ]:
# Definition einer Pipeline mit Preprocessing und LinearRegression-Modell
linear_pipe = Pipeline([
    ('preprocessing', column_trans),  # Vorverarbeitung der Eingabedaten
    ('model', LinearRegression())    # Lineares Regressionsmodell
])

# Parameter-Grid für das Modell (leer, da keine Hyperparameter optimiert werden)
linear_param_grid = {}

# GridSearchCV zur Optimierung des Modells
linear_gs = GridSearchCV(
    estimator=linear_pipe,                    # Pipeline als Estimator
    param_grid=linear_param_grid,             # Parameter-Grid
    scoring='neg_root_mean_squared_error',    # Bewertungsmetrik (negativer RMSE)
    cv=5,                                     # Kreuzvalidierung mit 5 Folds
    n_jobs=-1                                 # Parallele Verarbeitung
)

# Fit des Modells auf die Trainingsdaten
linear_gs.fit(X_train, y_train)

# Bestes Modell und Trainings-RMSE (basierend auf Kreuzvalidierung)
linear_best_model = linear_gs.best_estimator_
linear_train_rmse = np.sqrt(-linear_gs.best_score_)

# Vorhersage auf dem Test-Datensatz und Berechnung des Test-RMSE
y_pred = linear_best_model.predict(X_test)
linear_test_rmse = root_mean_squared_error(y_test, y_pred)

# Ausgabe der Ergebnisse
print("Bestes RMSE auf dem Trainingsdatensatz (Kreuzvalidierung):", linear_train_rmse)
print("RMSE auf dem Testdatensatz:", linear_test_rmse)


#### Die wichtigsten Features (Einfache lineare Regression)

In [ ]:
# Zugriff auf das LinearRegression-Modell aus der Pipeline
linear_model = linear_best_model.named_steps['model']

# Extrahieren der Feature-Namen aus dem Preprocessing-Schritt
if hasattr(linear_best_model.named_steps['preprocessing'], 'get_feature_names_out'):
    feature_names = linear_best_model.named_steps['preprocessing'].get_feature_names_out()
else:
    feature_names = X_train.columns

# Erstellung eines DataFrames mit den Koeffizienten und den entsprechenden Feature-Namen
coefficients_linear = pd.DataFrame({
    "Feature Name": feature_names,
    "Coefficient": linear_model.coef_
})

# Sortieren der Koeffizienten nach ihrem absoluten Wert in absteigender Reihenfolge
coefficients_linear_sorted = coefficients_linear.sort_values("Coefficient", key=abs, ascending=False)

# Ausgabe der Top-Features nach der Höhe ihrer Koeffizienten
print(coefficients_linear_sorted.head())


### Ridge()

In [ ]:
# Definition einer Pipeline mit Preprocessing und Ridge-Regression
ridge_pipe = Pipeline([
    ('preprocessing', column_trans),  # Vorverarbeitung der Daten
    ('model', Ridge())                # Ridge-Regression
])

# Parameter-Grid für die Ridge-Regression (Optimierung von Alpha(lambda))
ridge_param_grid = {"model__alpha": [0.01, 0.1, 1.0, 10.0, 100.0]}

# GridSearchCV zur Optimierung des Modells
ridge_gs = GridSearchCV(
    estimator=ridge_pipe,                    # Pipeline als Estimator
    param_grid=ridge_param_grid,             # Parameter-Grid
    scoring='neg_root_mean_squared_error',   # Bewertungsmetrik (negativer RMSE)
    cv=5,                                    # Kreuzvalidierung mit 5 Folds
    n_jobs=-1                                # Parallele Verarbeitung
)

# Fit des Modells auf die Trainingsdaten
ridge_gs.fit(X_train, y_train)

# Bestes Modell, beste Parameter und Trainings-RMSE (basierend auf Kreuzvalidierung)
ridge_best_params = ridge_gs.best_params_
ridge_best_model = ridge_gs.best_estimator_
ridge_train_rmse = np.sqrt(-ridge_gs.best_score_)

# Vorhersage auf dem Test-Datensatz und Berechnung des Test-RMSE
y_pred = ridge_best_model.predict(X_test)
ridge_test_rmse = root_mean_squared_error(y_test, y_pred)

# Ausgabe der Ergebnisse
print("Beste Parameter:", ridge_best_params)
print("Bestes RMSE auf dem Trainingsdatensatz (Kreuzvalidierung):", ridge_train_rmse)
print("RMSE auf dem Testdatensatz:", ridge_test_rmse)


#### Die wichtigsten Features (Ridge)

In [ ]:
# Zugriff auf das Ridge-Modell aus der Pipeline
ridge_model = ridge_best_model.named_steps['model']

# Extrahieren der Feature-Namen aus dem Preprocessing-Schritt
if hasattr(ridge_best_model.named_steps['preprocessing'], 'get_feature_names_out'):
    feature_names = ridge_best_model.named_steps['preprocessing'].get_feature_names_out()
else:
    feature_names = X_train.columns

# Erstellung eines DataFrames mit den Koeffizienten und den entsprechenden Feature-Namen
coefficients_ridge = pd.DataFrame({
    "Feature Name": feature_names,
    "Coefficient": ridge_model.coef_
})

# Sortieren der Koeffizienten nach ihrem absoluten Wert in absteigender Reihenfolge
coefficients_ridge_sorted = coefficients_ridge.sort_values("Coefficient", key=abs, ascending=False)

# Ausgabe der Top-Features nach der Höhe ihrer Koeffizienten
print(coefficients_ridge_sorted.head())


### Lasso()

In [ ]:
# Definition einer Pipeline mit Preprocessing und Lasso-Regression
lasso_pipe = Pipeline([
    ('preprocessing', column_trans),  # Vorverarbeitung der Eingabedaten
    ('model', Lasso())                # Lasso-Regression
])

# Parameter-Grid für die Lasso-Regression (Optimierung von Alpha(lambda))
lasso_param_grid = {
    "model__alpha": [0.01, 0.1, 1.0, 10.0, 100.0]
}

# GridSearchCV zur Optimierung des Modells
lasso_gs = GridSearchCV(
    estimator=lasso_pipe,                    # Pipeline als Estimator
    param_grid=lasso_param_grid,             # Parameter-Grid
    scoring='neg_root_mean_squared_error',   # Bewertungsmetrik (negativer RMSE)
    cv=5,                                    # Kreuzvalidierung mit 5 Folds
    n_jobs=-1                                # Parallele Verarbeitung
)

# Fit des Modells auf die Trainingsdaten
lasso_gs.fit(X_train, y_train)

# Bestes Modell, beste Parameter und Trainings-RMSE (basierend auf Kreuzvalidierung)
lasso_best_params = lasso_gs.best_params_
lasso_best_model = lasso_gs.best_estimator_
lasso_train_rmse = np.sqrt(-lasso_gs.best_score_)

# Vorhersage auf dem Test-Datensatz und Berechnung des Test-RMSE
y_pred = lasso_best_model.predict(X_test)
lasso_test_rmse = root_mean_squared_error(y_test, y_pred)

# Ausgabe der Ergebnisse
print("Beste Parameter:", lasso_best_params)
print("Bestes RMSE auf dem Trainingsdatensatz (Kreuzvalidierung):", lasso_train_rmse)
print("RMSE auf dem Testdatensatz:", lasso_test_rmse)


#### Die wichtigsten Features (Lasso)

In [ ]:
# Zugriff auf das Lasso-Modell aus der Pipeline
lasso_model = lasso_best_model.named_steps['model']

# Extrahieren der Feature-Namen aus dem Preprocessing-Schritt
if hasattr(lasso_best_model.named_steps['preprocessing'], 'get_feature_names_out'):
    feature_names = lasso_best_model.named_steps['preprocessing'].get_feature_names_out()
else:
    feature_names = X_train.columns

# Erstellung eines DataFrames mit den Koeffizienten und den entsprechenden Feature-Namen
coefficients_lasso = pd.DataFrame({
    "Feature Name": feature_names,
    "Coefficient": lasso_model.coef_
})

# Filtern der Features mit Koeffizienten ungleich null (nicht eliminierte Features)
non_zero_coefficients = coefficients_lasso[coefficients_lasso["Coefficient"] != 0]

# Sortieren der nicht-null Koeffizienten nach ihrem absoluten Wert in absteigender Reihenfolge
coefficients_lasso_sorted = non_zero_coefficients.sort_values("Coefficient", key=abs, ascending=False)

# Ausgabe der Top-Features nach der Höhe ihrer Koeffizienten
print(coefficients_lasso_sorted.head())


### Entscheidungsbaum

In [ ]:
# Definition einer Pipeline mit Preprocessing und DecisionTreeRegressor
dt_pipe = Pipeline([
    ('preprocessing', column_trans),              # Vorverarbeitung der Eingabedaten
    ('model', DecisionTreeRegressor(random_state=42))  # Entscheidungsbaum-Regressor
])

# Parameter-Grid für den Entscheidungsbaum (maximale Tiefe und minimale Anzahl an Proben für eine Teilung)
dt_param_grid = {
    "model__max_depth": [3, 5, 7, 10, 20, 25, 50, None],
    "model__min_samples_split": [2, 5, 10, 15, 20, 30, 50, 100]
}

# GridSearchCV zur Optimierung des Modells
dt_gs = GridSearchCV(
    estimator=dt_pipe,                    # Pipeline als Estimator
    param_grid=dt_param_grid,             # Parameter-Grid
    scoring='neg_root_mean_squared_error',  # Bewertungsmetrik (negativer RMSE)
    cv=5,                                 # Kreuzvalidierung mit 5 Folds
    n_jobs=-1                             # Parallele Verarbeitung
)

# Fit des Modells auf die Trainingsdaten
dt_gs.fit(X_train, y_train)

# Bestes Modell, beste Parameter und Trainings-RMSE (basierend auf Kreuzvalidierung)
dt_best_params = dt_gs.best_params_
dt_best_model = dt_gs.best_estimator_
dt_train_rmse = np.sqrt(-dt_gs.best_score_)

# Ausgabe der besten Parameter und des RMSE auf dem Trainingsdatensatz
print("Decision Tree Best Parameters:", dt_best_params)
print("Decision Tree Best RMSE (Cross-Validation):", dt_train_rmse)

# Vorhersage auf dem Test-Datensatz und Berechnung des Test-RMSE
y_pred = dt_best_model.predict(X_test)
dt_test_rmse = root_mean_squared_error(y_test, y_pred)

# Ausgabe des Test-RMSE
print("\nTest RMSE:", dt_test_rmse)


#### Die wichtigsten Features (Entscheidungsbaum)

In [ ]:
# Zugriff auf das Decision Tree-Modell aus der Pipeline
dt_model = dt_best_model.named_steps['model']

# Extrahieren der Feature-Namen aus dem Preprocessing-Schritt
if hasattr(dt_best_model.named_steps['preprocessing'], 'get_feature_names_out'):
    feature_names = dt_best_model.named_steps['preprocessing'].get_feature_names_out()
else:
    feature_names = X_train.columns

# Visualisierung des Entscheidungsbaums
plt.figure(figsize=(20, 10))  # Festlegen der Größe der Grafik
plot_tree(
    dt_model,
    feature_names=feature_names,  # Anzeige der Feature-Namen
    rounded=True,                 # Abgerundete Knoten
    filled=True,                  # Farbige Knoten basierend auf den Werten
    fontsize=10                   # Schriftgröße
)
plt.title("Decision Tree Visualization", fontsize=16)  # Titel der Grafik
plt.show()


### Ensemble-Modell(Gradient Boosting)

In [ ]:
# Definition einer Pipeline mit Preprocessing und GradientBoostingRegressor
pipe = Pipeline([
    ("preprocessing", column_trans),                     # Vorverarbeitung der Eingabedaten
    ("model", GradientBoostingRegressor(random_state=42))  # Gradient Boosting Regressor
])

# Parameter-Grid für die Optimierung des Modells
param_grid = {
    "model__learning_rate": [0.01, 0.1, 0.2],             # Lernrate
    "model__n_estimators": [100, 200, 500],               # Anzahl der Bäume
    "model__max_depth": [3, 5, 10],                       # Maximale Tiefe der Bäume
    "model__max_features": ["sqrt", "log2", None],        # Anzahl der zu verwendenden Features
    "preprocessing__onehot__impute__strategy": ["most_frequent"],  #  für kategorische Variablen
    "preprocessing__impute_scale__impute__strategy": ["mean"]    # für numerische Variablen
}

# GridSearchCV zur Optimierung des Modells
gs = GridSearchCV(estimator=pipe, param_grid=param_grid, scoring="neg_root_mean_squared_error", cv=5, n_jobs=-1)

# Fit des Modells auf die Trainingsdaten
gs.fit(X_train, y_train)

# Bestes Modell, beste Parameter und Trainings-RMSE (basierend auf Kreuzvalidierung)
best_params = gs.best_params_
best_model = gs.best_estimator_
train_rmse = np.sqrt(-gs.best_score_)

# Ausgabe der besten Parameter und des RMSE auf dem Trainingsdatensatz
print("Best Parameters:", best_params)
print("Best RMSE (Cross-Validation):", train_rmse)


In [ ]:
# Vorhersage auf dem Test-Datensatz
y_pred = best_model.predict(X_test)

# Berechnung des RMSE auf dem Test-Datensatz
test_rmse = root_mean_squared_error(y_test, y_pred)

# Ausgabe der Ergebnisse
print("Best Parameters:", best_params)
print("Best RMSE on Training Set (Cross-Validation):", train_rmse)
print("Test RMSE:", test_rmse)


#### Die wichtigsten Features (Gradient Boosting)

In [ ]:
# Zugriff auf das beste Modell aus der Pipeline
best_model = gs.best_estimator_

# Zugriff auf den GradientBoostingRegressor aus der Pipeline
gbr_model = best_model.named_steps['model']

# Extrahieren der Feature-Namen aus dem Preprocessing-Schritt
if hasattr(best_model.named_steps['preprocessing'], 'get_feature_names_out'):
    feature_names = best_model.named_steps['preprocessing'].get_feature_names_out()
else:
    feature_names = X_train.columns

# Zugriff auf die Feature-Importances des GradientBoostingRegressor
feature_importances = gbr_model.feature_importances_

# Erstellung eines DataFrames mit den Importances und den entsprechenden Feature-Namen
importances_df = pd.DataFrame({
    "Feature Name": feature_names,
    "Importance": feature_importances
})

# Sortieren der Features nach ihrer Wichtigkeit in absteigender Reihenfolge
importances_sorted = importances_df.sort_values("Importance", ascending=False)

# Ausgabe der Top-Features
print("Top-Features nach Wichtigkeit:")
print(importances_sorted.head())

# Optional: Visualisierung der Feature-Importances
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(importances_sorted["Feature Name"].head(10), importances_sorted["Importance"].head(10))
plt.xlabel("Feature Importance")
plt.ylabel("Feature Name")
plt.title("Top 10 wichtigste Features")
plt.gca().invert_yaxis()  # Damit die wichtigsten Features oben stehen
plt.show()
